In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Feb 16 09:13:21 2022
@author: ismail.oubarka
"""

from mpi4py import MPI
import timeit

COMM = MPI.COMM_WORLD
SIZE = COMM.Get_size()
RANK = COMM.Get_rank()

from manapy.ddm import readmesh
from manapy.ddm import Domain

from manapy.tools.pyccel_tools import initialisation_gaussian_2d, update_new_value, time_step
                          
from manapy.fvm.pyccel_fvm import (explicitscheme_convective_2d,
                                   explicitscheme_dissipative)

#from manapy.ast import Variable, LinearSystem

import numpy as np
import os

start = timeit.default_timer()

# mesh directory

dim = 2
readmesh('/home/ayoub.daoudia/Masters/Advanced Computational Methods/S2/TP/test2.msh', dim=dim, periodic=[0,0,0])

#Create the informations about cells, faces and nodes
domain = Domain(dim=dim)

faces = domain.faces
cells = domain.cells
halos = domain.halos
nodes = domain.nodes

nbnodes = domain.nbnodes
nbfaces = domain.nbfaces
nbcells = domain.nbcells


Starting ....
Number of Cells :  4636
Number of Nodes :  2409


In [3]:
domain.nbcells

4636

In [4]:
#Cells Centre
Cells_centre= [cell[:-1] for cell in cells._center]

In [5]:
import numpy.linalg as LA

def circumcenter(C):
    ax = C[0][0]
    ay = C[0][1]
    bx = C[1][0]
    by = C[1][1]
    cx = C[2][0]
    cy = C[2][1]
    d = 2 * (ax * (by - cy) + bx * (cy - ay) + cx * (ay - by))
    ux = ((ax * ax + ay * ay) * (by - cy) + (bx * bx + by * by) * (cy - ay) + (cx * cx + cy * cy) * (ay - by)) / d
    uy = ((ax * ax + ay * ay) * (cx - bx) + (bx * bx + by * by) * (ax - cx) + (cx * cx + cy * cy) * (bx - ax)) / d
    return (ux, uy)

def dist(x, y):
    x=np.asarray(x)
    y=np.asarray(y)
    return LA.norm(x - y)

In [6]:
def norm2(exact, Sol, volume, nbelements, order):      
    Error = np.zeros(nbelements)
    Ex = np.zeros(nbelements)
    for i in range(nbelements):
        Error[i] = np.fabs(Sol[i]- exact[i]) * volume[i]
        Ex[i] = np.fabs(exact[i]) * volume[i]

    ErrorL2 = np.linalg.norm(Error,ord=order)/np.linalg.norm(Ex,ord=order)
    return ErrorL2

In [7]:
def cell_ver_coor():
    Cells_Cord=[]
    for s in cells._nodeid :
        Cell_Cord =[]
        for i in s :
            Cell_Cord.append(nodes._vertex[i][:-2])
        Cells_Cord.append(Cell_Cord)   
    return Cells_Cord

Cells_Cord = cell_ver_coor()

In [8]:
#Orthocentre
Orthocentre = []
for i in range(nbcells):
    Orthocentre.append(circumcenter(Cells_Cord[i]))

In [9]:
 def save_paraview_one_variable(w, cells, nodes, dim, name):
    
    if dim == 2:
        elements = {"triangle": cells}
    elif dim == 3:
        elements = {"tetra": cells}

    points = []
    for i in nodes:
        points.append([i[0], i[1], i[2]])
    
    cells  = np.array(cells)
    points = np.array(points)
    
    data = {"w_v1" : w}
   
    if len(w) == len(cells):
        data = {"w_v1": data}

    if len(w) == len(cells):
        meshio.write_points_cells("results_2_v1/visu"+name+".vtu",
                                  points, elements, cell_data=data, file_format="vtu")
    else:
        meshio.write_points_cells("results_2_v1/visu"+str(w)+".vtu",
                                  points, elements, point_data=data, file_format="vtu")


In [10]:
dist(Orthocentre[0],Orthocentre[1])

104.3231471212392

In [11]:
np.unique(nodes.vertex[:,3])

array([0., 2., 3., 4., 5.])

In [12]:
cells_dict = {"Bottom":[],
             "Right":[],
             "Top":[],
             "Left":[]}
for i in range(nbcells):
    for j in range(3):
        if nodes.vertex[cells.nodeid[i][j]][3] == 2:
#             print("Bottom", i)
            cells_dict["Bottom"] += [i]
            break
        elif nodes.vertex[cells.nodeid[i][j]][3] == 3:
#             print("Right", i)
            cells_dict["Right"] += [i]
            break
        elif nodes.vertex[cells.nodeid[i][j]][3] == 4:
#             print("Top", i)
            cells_dict["Top"] += [i]
            break
        elif nodes.vertex[cells.nodeid[i][j]][3] == 5:
#             print("Left", i)
            cells_dict["Left"] += [i]
            break


In [176]:
import meshio
import numpy as np
def Sol_init():
    return 0


u_init = []
for i in range(nbcells):
    if i in (cells_dict["Bottom"] or cells_dict["Right"]):
        u_init += [0]
    elif i in cells_dict["Top"]:
        u_init += [10]
    elif i in cells_dict["Left"]:
        u_init += [20]
    else :
        u_init += [Sol_init()]
u_init = np.array(u_init)


save_n=0
save_paraview_one_variable(u_init, cells._nodeid, nodes._vertex, 2, "res_VF4"+str(save_n))

In [177]:
cells.faceid[0]

array([4115, 4918, 4113])

In [178]:
cells.center[0]

array([ 4.84290593, 17.16035315,  0.        ])

In [179]:
target_cells=2
cell1 = faces.cellid[cells.faceid[target_cells]][0]
cell2 = faces.cellid[cells.faceid[target_cells]][1]
cell3 = faces.cellid[cells.faceid[target_cells]][2]

cells.faceid[target_cells],cell1,cell2,cell3

(array([ 809, 5514,  808]),
 array([ 2, 16]),
 array([   2, 1135]),
 array([ 2, 82]))

In [180]:
target_cells=0

cells.faceid[target_cells]

array([4115, 4918, 4113])

In [181]:
face=cells.faceid[target_cells]
k = faces.cellid[face][0]
l = faces.cellid[face][1]

face,k,l

(array([4115, 4918, 4113]), array([ 0, 29]), array([ 0, 36]))

In [182]:
mesure = faces.mesure[face]
mesure

array([2.2979952 , 2.38815291, 2.00925498])

In [183]:
dist1 = dist(Orthocentre[k[0]], Orthocentre[l[1]])
dist1

0.7482953295491088

In [184]:
faces.cellid[cells.faceid[1060][0]]

array([1060, 2983])

In [185]:
# Calcul de dt
somme = []
for i in range(nbcells):
    k = cells.volume[i]
    list_faces = list(cells.faceid[i])
    elem = 0
    for j in range(3):
        mesure1 = faces.mesure[list_faces[j]]
        dist1 = dist(Orthocentre[list(faces.cellid[list_faces[j]])[0]],Orthocentre[list(faces.cellid[list_faces[j]])[1]])
        elem += mesure1/dist1
    somme += [1/k*elem]

lamda = 0.8
D = 1
# Fin de calcul
dt = lamda/(D * max (somme))
print(dt)

save_n = 0
temps=0
T=50
u_v1=np.copy(u_init)
i=0
unew = np.zeros(nbcells)
while temps<=T:
    temps+=dt
    save_n+=1
    
    for j in range(nbcells):
        list_faces = cells.faceid[j]
        elem = 0
        for k in range(3):
            mesure1 = faces.mesure[list_faces[k]]
            dist1 = dist(Orthocentre[faces.cellid[list_faces[k]][0]],Orthocentre[faces.cellid[list_faces[k]][1]])
            if j == faces.cellid[list_faces[k]][0]:
                u_diff = u_v1[faces.cellid[list_faces[k]][1]] - u_v1[faces.cellid[list_faces[k]][0]]
            else :
                u_diff = u_v1[faces.cellid[list_faces[k]][0]] - u_v1[faces.cellid[list_faces[k]][1]]
            elem += mesure1*u_diff/dist1
        unew[j] = u_v1[j] - dt/(cells.volume[j]) * (-D)*elem
        if j in (cells_dict["Bottom"] or cells_dict["Right"]):
            unew[j] = 0
        elif j in cells_dict["Top"]:
            unew[j] = 10
        elif j in cells_dict["Left"]:
            unew[j] = 20

    u_v1=np.copy(unew)
    i+=1
    if i % 100 == 0:
        print(i, temps)
    save_paraview_one_variable(u_v1, cells._nodeid, nodes._vertex, 2, "res_VF4"+str(save_n))

0.11934628745173964
100 11.934628745173951
200 23.86925749034797
300 35.80388623552205
400 47.73851498069613


In [186]:
print(i,temps)

419 50.006094442279206


In [13]:
 def save_paraview_one_variable(w, cells, nodes, dim, name):
    
    if dim == 2:
        elements = {"triangle": cells}
    elif dim == 3:
        elements = {"tetra": cells}

    points = []
    for i in nodes:
        points.append([i[0], i[1], i[2]])
    
    cells  = np.array(cells)
    points = np.array(points)
    
    data = {"w" : w}
   
    if len(w) == len(cells):
        data = {"w": data}

    if len(w) == len(cells):
        meshio.write_points_cells("results_2_v2/visu"+name+".vtu",
                                  points, elements, cell_data=data, file_format="vtu")
    else:
        meshio.write_points_cells("results_2_v2/visu"+str(w)+".vtu",
                                  points, elements, point_data=data, file_format="vtu")


In [14]:
import meshio
import numpy as np
def Sol_init():
    return 0


u_init = []
for i in range(nbcells):
    if i in (cells_dict["Bottom"] or cells_dict["Right"]):
        u_init += [0]
    elif i in cells_dict["Top"]:
        u_init += [10]
    elif i in cells_dict["Left"]:
        u_init += [20]
    else :
        u_init += [Sol_init()]
u_init = np.array(u_init)


save_n=0
save_paraview_one_variable(u_init, cells._nodeid, nodes._vertex, 2, "res_VF4"+str(save_n))

In [15]:
def centroid(C):
    ax = C[0][0]
    ay = C[0][1]
    bx = C[1][0]
    by = C[1][1]
    cx = C[2][0]
    cy = C[2][1]
    return ((ax+bx+cx)/3,(ay+by+cy)/3)

Barycentre = []
for i in range(nbcells):
    Barycentre.append(centroid(Cells_Cord[i]))

In [16]:
# Faces_Cord[921],Faces_Cord[1636],Faces_Cord[917], cells.faceid[0]

In [17]:
# faces.f_1[0], faces.f_2[0], faces.f_3[0], faces.f_4[0], faces.center[0],Cells_Cord[0]

In [18]:
# faces.center[cells.faceid[0][0],:-1]
faces.cellid[1]
# cells.faceid[543]
# cells.faceid[0][0]
# nodes.vertex[10]
# faces.nodeid[0]

array([4515,   -1])

In [19]:
def face_ver_coor():
    Faces_Cord=[]
    for s in faces._nodeid :
        Face_Cord =[]
        for i in s :
            Face_Cord.append(nodes._vertex[i][:-2])
        Faces_Cord.append(Face_Cord)   
    return Faces_Cord

Faces_Cord = face_ver_coor()

In [20]:
def projection_point_on_line(point, line_start, line_end):
    line_vec = np.array(line_end) - np.array(line_start)
    point_vec = np.array(point) - np.array(line_start)
    line_len = np.linalg.norm(line_vec)
    line_unitvec = line_vec / line_len
    point_vec_scaled = point_vec / line_len
    t = np.dot(line_unitvec, point_vec_scaled)
    t = np.clip(t, 0, 1)  # Ensure the projection is on the line segment
    nearest = np.array(line_start) + t * line_vec
    return nearest

In [21]:
def dist_bary_edge(bary, edge_vertices):
    edge_start, edge_end = edge_vertices
    proj_point = projection_point_on_line(bary, edge_start, edge_end)
    return dist(bary, proj_point)

In [22]:
Faces_Cord[921],Faces_Cord[1636],Faces_Cord[917], cells.faceid[0]

([array([40.        , 59.21539031]), array([41.       , 57.4833395])],
 [array([48.        , 24.57437416]), array([49.        , 22.84232335])],
 [array([38.        , 59.21539031]), array([39.       , 57.4833395])],
 array([4115, 4918, 4113]))

In [ ]:
# Calcul de dt
somme = []
for i in range(nbcells):
    k = cells.volume[i]
    list_faces = list(cells.faceid[i])
    elem = 0
    for j in range(3):
        mesure1 = faces.mesure[list_faces[j]]
        dist1 = dist(Orthocentre[list(faces.cellid[list_faces[j]])[0]],Orthocentre[list(faces.cellid[list_faces[j]])[1]])
        elem += mesure1/dist1
    somme += [1/k*elem]

lamda = 0.8
D = 1
# Fin de calcul
dt = lamda/(D * max (somme))
print(dt)

save_n = 0
temps=0
T=100
u_v2=np.copy(u_init)
i=0
unew = np.zeros(nbcells)
while temps<=T:
    temps+=dt
    save_n+=1
    
    for j in range(nbcells):
        list_faces = cells.faceid[j]
        elem = 0
        for k in range(3):
            idcell1 = faces.cellid[list_faces[k]][0]
            idcell2 = faces.cellid[list_faces[k]][1]
            mesure1 = faces.mesure[list_faces[k]]
#             dist1 = dist(Orthocentre[faces.cellid[list_faces[k]][0]],Orthocentre[faces.cellid[list_faces[k]][1]])
            dist1 = dist_bary_edge(Barycentre[idcell1],Faces_Cord[list_faces[k]])
            dist2 = dist_bary_edge(Barycentre[idcell2],Faces_Cord[list_faces[k]])
            if j == faces.cellid[list_faces[k]][0]:
                u_diff = u_v2[faces.cellid[list_faces[k]][1]] - u_v2[faces.cellid[list_faces[k]][0]]
            else :
                u_diff = u_v2[faces.cellid[list_faces[k]][0]] - u_v2[faces.cellid[list_faces[k]][1]]
            elem += mesure1*u_diff/dist1
        unew[j] = u_v2[j] - dt/(cells.volume[j]) * (-D)*elem
        if j in (cells_dict["Bottom"] or cells_dict["Right"]):
            unew[j] = 0
        elif j in cells_dict["Top"]:
            unew[j] = 10
        elif j in cells_dict["Left"]:
            unew[j] = 20

    u_v2=np.copy(unew)
    i+=1
    if i % 100 == 0:
        print(i,temps)
    save_paraview_one_variable(u_v2, cells._nodeid, nodes._vertex, 2, "res_VF4"+str(save_n))

0.11934628745173964
100 11.934628745173951


In [ ]:
#####sudo apt-get install paraview